## init ##

In [1]:
import itertools
from collections import defaultdict, Counter
from functools import reduce
import re
import nltk
from math import log2
import pandas as pd
from Bio import SeqIO, SeqRecord, Seq
from tqdm import tqdm_notebook as tqdm
import concurrent.futures 
import sys, os
import glob
from functools import partial
import pickle
from operator import itemgetter as ig
import requests
import json
from time import sleep
import goatools
import wget

In [2]:
def _L(x):
    return list(x)
    
def _L0(x):
    return list(x)[0]


In [3]:
def grouper(n, iterable):
    it = iter(iterable)
    while True:
        chunk = tuple(itertools.islice(it, n))
        if not chunk:
            return
        yield chunk


In [4]:
def filter_desc(desc, split = False):
    unlist = False
    if isinstance(desc, str):
        desc = [desc]
        unlist = True
        
    desc_out = []
    
    for s in desc:
        s = re.sub(r'(\[.+\])|(\(.+\))', '', s)
        s = re.sub(r'gi\|.+', '', s)
        s = s.lower()
        if not split:
            desc_out.append(s)
        else:
            desc_out.extend(s.split('\n'))
                
        
    if unlist and not split:
        desc_out = desc_out[0]
    
    desc_out = map(str.strip, desc_out)
    desc_out = filter(None, desc_out)
    return list(desc_out)

In [5]:
def ngrams(s):
    res = []
    wlist = s.split()
    for clen in range(1, len(wlist)+1):
        for shift in range(0, len(wlist)-clen+1):
            res.append(" ".join(wlist[shift:clen+shift]))

    return res

In [6]:
def most_freq_substr(strings, oneres = True):
    ngr = []
    for s in strings:
        ngr.extend(ngrams(s))
    res = sorted(Counter(ngr).items(), key = lambda x: x[1]*len(x[0]), reverse=True)
    return res[0][0] if oneres else res

In [7]:
go_obo_url = 'http://purl.obolibrary.org/obo/go/go-basic.obo'
data_folder = os.getcwd() + '/data'

# Check if we have the ./data directory already
if(not os.path.isfile(data_folder)):
    # Emulate mkdir -p (no error if folder exists)
    try:
        os.mkdir(data_folder)
    except OSError as e:
        if(e.errno != 17):
            raise e
else:
    raise Exception('Data path (' + data_folder + ') exists as a file. '
                   'Please rename, remove or change the desired location of the data path.')

# Check if the file exists already
if(not os.path.isfile(data_folder+'/go-basic.obo')):
    go_obo = wget.download(go_obo_url, data_folder+'/go-basic.obo')
else:
    go_obo = data_folder+'/go-basic.obo'

go = goatools.obo_parser.GODag(go_obo)

load obo file /home/urban/pepf_blast/data/go-basic.obo
/home/urban/pepf_blast/data/go-basic.obo: fmt(1.2) rel(2017-10-21) 47,002 GO Terms


 ## Parse peptbast output ##

In [ ]:
%cd ~/pepf_blast/

In [8]:
def parse_peptblast(filename):
    for ch in grouper(7, open(filename).readlines()):
        record = {}
        record['q'] = ch[1].split("\t")[0]
        record['h'] = ch[5].split("\t")[0]
        record['h-desc'] = ch[5].split("\t")[2]
        record['_'] = ch
        yield record        

In [ ]:
descs = defaultdict(list)
for r in parse_peptblast("peps1740.fasta_NCBInr_20160328.fasta_1.txt"):
    descs[r['q']].append(r['h-desc'])

In [ ]:
all_descs = {k: "".join(v) for k, v in descs.items()}

In [ ]:
gids = [r['h'] for r in parse_peptblast("peps1740.fasta_NCBInr_20160328.fasta_1.txt")]
gids = [g.split('|')[1] for g in gids]

In [ ]:
gogids = {l.split('\t')[1] for l in open("gene2go").readlines()}

In [ ]:
gogids.intersection(gids)

In [ ]:
all_descs['all1539']

In [ ]:
descs1 = {k:most_freq_substr(filter_desc(v, True)) for k,v in descs.items()}

In [ ]:
descs1

In [ ]:
len({p['q'] for p in parse_peptblast("peps1740.fasta_NCBInr_20160328.fasta_1.txt") })

In [ ]:
len({p['q'] for p in parse_peptblast("peps1740.fasta_NCBInr_20160328.fasta_1.txt") 
    if 'hypothetical' not in p['h-desc'].lower()})

In [ ]:
sum( map(lambda x: any( map(lambda y: y in x,
                            ['membrane', 'transporter', 'permease', 'envelope', 'atp syntase']) ), descs1.values()))

In [ ]:
sum( map(lambda x: any( map(lambda y: y in x, ['ribosom']) ), descs1.values()))

## Read plain blast ##

In [ ]:
blast = [(l.split('\t')[0],l.split('\t')[1]) for l in open("peps1740_nr_result.txt").readlines()]

## Stupidly find pep sequences in HuBaCon ##

In [7]:
peps = [(r.name,str(r.seq)) for r in SeqIO.parse("peps1740.fasta", "fasta")]

In [229]:
n_huba = !grep '>' MICROBIOTA_new_unique_cd_hit_sim_0_9.fasta | wc -l
n_huba = int(n_huba[0])

In [ ]:
found = defaultdict(list)
for r in tqdm(SeqIO.parse("MICROBIOTA_new_unique_cd_hit_sim_0_9.fasta", "fasta"), total=n_huba):
    for pn, ps in peps:
        if ps.lower() in str(r.seq).lower():
            found[pn].append(r.name)

In [ ]:
#pickle.dump(found, open("1740_in_hubacon_stupid.pkl", "wb"))

In [ ]:
peps

In [ ]:
len(found.keys()), len(peps)

In [ ]:
list(filter(lambda x: x[1]!=1, map(lambda x: (x[0], len(x[1])), found)))

In [ ]:
set(map(lambda x: x[0], peps)).difference(found.keys())

In [ ]:
list(map(lambda x: (x[0], x[1].lower()), filter(lambda x: x[0] in _26, peps)))

In [ ]:
found

In [ ]:
chunksize = 10000
if not os.path.exists("nr_split"):
    os.mkdir("nr_split")
for n,r in enumerate(grouper(chunksize, tqdm(SeqIO.parse("NCBInr_20160328.fasta", "fasta"), total=83911944))):
    with open("nr_split/%05d.fasta" % n, "w") as fo:
        SeqIO.write(r, fo, "fasta")

HBox(children=(IntProgress(value=0, max=83911944), HTML(value='')))

In [231]:
chunksize = 100000
if not os.path.exists("mcc_split"):
    os.mkdir("mcc_split")
for n,r in enumerate(grouper(chunksize, tqdm(SeqIO.parse("MICROBIOTA_new_unique_cd_hit_sim_0_9.fasta", "fasta"), total=n_huba))):
    with open("mcc_split/%05d.fasta" % n, "w") as fo:
        SeqIO.write(r, fo, "fasta")

HBox(children=(IntProgress(value=0, max=1832593), HTML(value='')))

In [8]:
bases = glob.glob("nr_split/*.fasta")

In [9]:
def find_peps(peps, fname):
    found = defaultdict(list)
    for r in SeqIO.parse(fname, "fasta"):
        for pn, ps in peps:
            if ps.lower() in str(r.seq).lower():
                found[pn].append(r)
   # print(fname+" done!")
    return found

In [10]:
def _find_peps(a):
    return find_peps(a[0], a[1])

In [11]:
from concurrent.futures import ProcessPoolExecutor, as_completed

def parallel_process(array, function, n_jobs=16, use_kwargs=False, front_num=3):
    """
        A parallel version of the map function with a progress bar. 

        Args:
            array (array-like): An array to iterate over.
            function (function): A python function to apply to the elements of array
            n_jobs (int, default=16): The number of cores to use
            use_kwargs (boolean, default=False): Whether to consider the elements of array as dictionaries of 
                keyword arguments to function 
            front_num (int, default=3): The number of iterations to run serially before kicking off the parallel job. 
                Useful for catching bugs
        Returns:
            [function(array[0]), function(array[1]), ...]
    """
    #We run the first few iterations serially to catch bugs
    if front_num > 0:
        front = [function(**a) if use_kwargs else function(a) for a in array[:front_num]]
    #If we set n_jobs to 1, just run a list comprehension. This is useful for benchmarking and debugging.
    if n_jobs==1:
        return front + [function(**a) if use_kwargs else function(a) for a in tqdm(array[front_num:])]
    #Assemble the workers
    with ProcessPoolExecutor(max_workers=n_jobs) as pool:
        #Pass the elements of array into function
        if use_kwargs:
            futures = [pool.submit(function, **a) for a in array[front_num:]]
        else:
            futures = [pool.submit(function, a) for a in array[front_num:]]
        kwargs = {
            'total': len(futures),
            'unit': 'it',
            'unit_scale': True,
            'leave': True
        }
        #Print out the progress as tasks complete
        for f in tqdm(as_completed(futures), **kwargs):
            pass
    out = []
    #Get the results from the futures. 
    for i, future in tqdm(enumerate(futures)):
        try:
            out.append(future.result())
        except Exception as e:
            out.append(e)
    return front + out

In [ ]:
ress = parallel_process(list(itertools.zip_longest([peps], bases, fillvalue=peps)),  _find_peps, n_jobs=32)

HBox(children=(IntProgress(value=0, max=8389), HTML(value='')))

HBox(children=(IntProgress(value=1, bar_style='info', max=1), HTML(value='')))

In [31]:
len(ress)

8392

In [35]:
pickle.dump(ress, open("_1_1740_in_nr_stupid.pkl", "wb"))

In [41]:
ress_all = defaultdict(list)
for r in ress:
    for k, v in r.items():
        ress_all[k].extend(v)

In [42]:
len(ress_all.keys())

1735

## New validation ##

In [21]:
new = pd.read_table("Spectrum-report_HuBaCon-90_afterPiP.csv")

In [22]:
new.head()

,Experiment name,Biological sample category,Biological sample name,MS/MS sample name,Protein name,Protein accession numbers,Database sources,Protein molecular weight (Da),Protein identification probability,Exclusive unique peptide count,...,Calculated +1H Peptide Mass (AMU),Spectrum charge,Actual minus calculated peptide mass (AMU),Actual minus calculated peptide mass (PPM),Retention Time (sec),Total Ion Current,Peptide start index,Peptide stop index,Exclusive,Other Proteins
0,HealthyDonors_Serum-Plasma_HuBaCon-90_72_2_FINAL,Healthy_Serum,BioGel,I305_MGFPeaklist,gi|490748864|ref|WP_004611172.1|,gi|490748864|ref|WP_004611172.1|,HuBaCon-90_20160703_TnRev.fasta,"33,867.5",89.5%,1,...,"1,031.584446",2,-0.003922,-3.8020,"1,731.00","18,903.1",262,271,True,NaN
1,HealthyDonors_Serum-Plasma_HuBaCon-90_72_2_FINAL,Healthy_Serum,BioGel,I305_MGFPeaklist,gi|497431592|ref|WP_009745790.1|,gi|497431592|ref|WP_009745790.1|,HuBaCon-90_20160703_TnRev.fasta,"83,651.0",99.8%,1,...,"1,362.726646",2,-0.020920,-15.3500,"3,048.00",1017450,132,143,True,NaN
2,HealthyDonors_Serum-Plasma_HuBaCon-90_72_2_FINAL,Healthy_Serum,BioGel,I305_MGFPeaklist,gi|547828915|ref|WP_022237044.1|,gi|547828915|ref|WP_022237044.1|,HuBaCon-90_20160703_TnRev.fasta,"28,344.6",76.7%,1,...,"1,106.551746",2,0.011180,10.1000,"2,650.00","20,397.4",39,49,True,NaN
3,HealthyDonors_Serum-Plasma_HuBaCon-90_72_2_FINAL,Healthy_Serum,BioGel,I305_MGFPeaklist,gi|737419315|ref|WP_035400192.1|,gi|737419315|ref|WP_035400192.1|,HuBaCon-90_20160703_TnRev.fasta,"75,657.5",52.3%,1,...,22.039674,5,4061.000000,7.0280,"3,350.00","35,293.9",0,0,True,NaN
4,HealthyDonors_Serum-Plasma_HuBaCon-90_72_2_FINAL,Healthy_Serum,BioGel,I305_MGFPeaklist,gi|496354965|ref|WP_009064141.1|,gi|496354965|ref|WP_009064141.1|,HuBaCon-90_20160703_TnRev.fasta,"66,093.4",61.4%,1,...,"1,967.128746",3,0.001202,0.6111,"2,685.00","76,584.0",128,144,True,NaN


In [23]:
seqs = list(new["Peptide sequence"])

In [24]:
seqs = sorted(list(set(map(str.upper, seqs))), key=len)

In [25]:
len(seqs)

737

In [26]:
recs = [SeqRecord.SeqRecord(seq=Seq.Seq(s, Seq.Alphabet.SingleLetterAlphabet), id="new%03d" % i) 
        for i,s in enumerate(seqs, 1)]
SeqIO.write(recs, "peps737.fasta", "fasta")    

737

In [42]:
chunksize = 10000
path = "mc_split"
filein = "MICROBIOTA_new_unique_cd_hit_sim_0_9.fasta"
n_tot = !grep '>' MICROBIOTA_new_unique_cd_hit_sim_0_9.fasta | wc -l
n_tot = int(n_tot[0])
if not os.path.exists(path):
    os.mkdir(path)
for n,r in enumerate(grouper(chunksize, tqdm(SeqIO.parse(filein, "fasta"), 
                                             total=n_tot))):
    with open(path+"/%05d.fasta" % n, "w") as fo:
        SeqIO.write(r, fo, "fasta")

HBox(children=(IntProgress(value=0, max=1832593), HTML(value='')))

In [206]:
from subprocess import getoutput
po = getoutput("grep '>' MICROBIOTA_new_unique_cd_hit_sim_0_9.fasta | wc -l")
int(po)

1832593

In [202]:
n

"b'1832593\\n'"

In [ ]:
_.

In [129]:
peps = [(r.name,str(r.seq)) for r in SeqIO.parse("peps737.fasta", "fasta")]

In [141]:
peps=[(n, s if n!="new724" else "fvcarlgggavlpxgggavlpladmktifagddtaali".upper())
      for n,s in peps]

In [142]:
peps

[('new001', 'PDLWQLLD'),
 ('new002', 'MHLIDDHA'),
 ('new003', 'PSLTSTPN'),
 ('new004', 'PPYNPVTL'),
 ('new005', 'DPIWQLID'),
 ('new006', 'VATVSIPR'),
 ('new007', 'LESYLDNV'),
 ('new008', 'PEDDYLSL'),
 ('new009', 'INMAEIAA'),
 ('new010', 'QVMAEIAA'),
 ('new011', 'LSESASLLR'),
 ('new012', 'HSMESASLI'),
 ('new013', 'VDFAQARES'),
 ('new014', 'PDLKGNQEV'),
 ('new015', 'IEVIPAVES'),
 ('new016', 'GHGIGMGGD'),
 ('new017', 'SHTIIVACL'),
 ('new018', 'LLEVPEAAA'),
 ('new019', 'REEENEELE'),
 ('new020', 'IGTIKLLSD'),
 ('new021', 'PDIKADLID'),
 ('new022', 'PLQIVVEDD'),
 ('new023', 'KENLLIYLD'),
 ('new024', 'PGPGPQGTP'),
 ('new025', 'FAHLIVEKF'),
 ('new026', 'DGIGWIPID'),
 ('new027', 'WPLTLAILL'),
 ('new028', 'YNELFGAFK'),
 ('new029', 'EDVQFADSR'),
 ('new030', 'PDISVELID'),
 ('new031', 'DFATSMGFD'),
 ('new032', 'KAGTVYDLP'),
 ('new033', 'PLDILVAES'),
 ('new034', 'PLKNSGVEF'),
 ('new035', 'LDVMQAPVI'),
 ('new036', 'EGDVIAFGK'),
 ('new037', 'PEELENMLL'),
 ('new038', 'MTVADFEAR'),
 ('new039', 'EDVQFADDK

In [143]:
bases = glob.glob("mc_split/*.fasta")

In [144]:
ress = parallel_process(list(itertools.zip_longest([], bases, fillvalue=peps)),  _find_peps, n_jobs=32)

HBox(children=(IntProgress(value=0, max=181), HTML(value='')))

HBox(children=(IntProgress(value=1, bar_style='info', max=1), HTML(value='')))

In [41]:
len(peps)

737

In [147]:
ress_all = defaultdict(list)
for r in ress:
    for k, v in r.items():
        ress_all[k].extend(v)

In [156]:
set(map(ig(0), peps)).difference(ress_all.keys())

set()

In [157]:
pickle.dump(ress_all, open("737_in_MB.pkl", "wb"))

In [13]:
mb737 = pickle.load(open("737_in_MB.pkl", "rb"))

In [16]:
len(mb737)

737

In [17]:
precs = [SeqRecord.SeqRecord(seq=v[0].seq, id=k+"_prec", name=v[0].id)
        for k,v in mb737.items()]

In [161]:
SeqIO.write(precs, "precs737.fasta", "fasta")

737

In [19]:
base_url = "http://eggnogapi.embl.de/nog_data/{dataformat}/{attributes}/{nogname}"
base_url_go = "http://eggnogapi.embl.de/nog_data/json/go_terms/{}"

In [20]:
requests.get(base_url.format(dataformat="html", attributes="go_terms", nogname="06RIU"))

<Response [200]>

In [21]:
eggnog = pd.read_table("precs737.fasta.emapper.annotations", header=None)

In [22]:
eggnog.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,new052_prec,944559.HMPREF9413_3815,1.300000e-197,651.0,RFBD,"GO:0000271,GO:0003674,GO:0003824,GO:0005575,GO...","K00067,K01790",TDPDRR,bactNOG[38],"04R51@bacNOG,05DBZ@bactNOG,0NE2K@firmNOG,COG10...",05DBZ|3.35972945553e-106|359.402374268,M,Dtdp-4-dehydrorhamnose reductase
1,new473_prec,908937.HMPREF9136_0887,1.500000e-143,467.6,NaN,NaN,NaN,NaN,bactNOG[38],"05972@bacteNOG,06JVU@bactNOG,09M4S@bctoNOG,0YD...",06JVU|1.2758379557e-58|201.685882568,G,Beta-galactosidase
2,new454_prec,717785.HYPMC_4854,9.600000e-109,351.9,AHPC,"GO:0003674,GO:0003824,GO:0004601,GO:0005575,GO...",K03386,NaN,bactNOG[38],"01TVQ@aproNOG,05D3R@bactNOG,16RND@proNOG,COG04...",01TVQ|7.60785778774e-106|354.77154541,O,alkyl hydroperoxide reductase subunit C
3,new246_prec,411471.SUBVAR_06444,6.600000e-179,589.0,MURI,"GO:0000270,GO:0003674,GO:0003824,GO:0004857,GO...","K01776,K02428",GLUR,bactNOG[38],"05F03@bactNOG,0EQ26@cloNOG,0NECR@firmNOG,COG07...",05F03|3.14243085682e-94|320.067321777,M,Provides the (R)-glutamate required for cell w...
4,new106_prec,85963.jhp0598,1.400000e-110,361.4,FTNA,"GO:0003674,GO:0003824,GO:0004322,GO:0005488,GO...",K02217,NaN,bactNOG[38],"090A1@bactNOG,0GDYV@delNOG,0I3JC@eproNOG,178EK...",090A1|1.93126537272e-62|212.668930054,P,ferritin


In [23]:
bests = eggnog.iloc[:,[0,10]]

bests.columns = "id", "nog"

In [24]:
bests.loc[:,"nog"] = bests.nog.map(lambda x: x.split("|")[0], 
                                  )

/home/urban/.local/lib/python3.4/site-packages/pandas/core/indexing.py:621: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item_labels[indexer[info_axis]]] = value


In [25]:
bests.iterrows().__next__()

(0, id     new052_prec
 nog          05DBZ
 Name: 0, dtype: object)

In [26]:
gos = dict()
for _, row in tqdm(bests.iterrows(), total = len(bests)):
    nrep = 5
    while nrep:
        sleep(1)
        try:
            gos[row[0]] = requests.get(base_url_go.format(row[1])).json()
        except Exception as e:
            nrep -= 1
            print (e)
        else:
            #print("break %s" % row[0])
            break
    

HBox(children=(IntProgress(value=0, max=662), HTML(value='')))

In [27]:
gos

{'new002_prec': {'go_header': ['ID',
   'GO term',
   'Evidence',
   'SeqCount',
   'Frequency',
   'relative_fontsize'],
  'go_terms': {'Biological Process': [['GO:0006826',
     'iron ion transport',
     'IEA',
     483,
     '96.8',
     '99.8'],
    ['GO:0044699', 'single-organism process', 'IEA', 483, '96.8', '99.8'],
    ['GO:0000041',
     'transition metal ion transport',
     'IEA',
     483,
     '96.8',
     '99.8'],
    ['GO:0051179', 'localization', 'IEA', 483, '96.8', '99.8'],
    ['GO:0006810', 'transport', 'IEA', 483, '96.8', '99.8'],
    ['GO:0006812', 'cation transport', 'IEA', 483, '96.8', '99.8'],
    ['GO:0006811', 'ion transport', 'IEA', 483, '96.8', '99.8'],
    ['GO:0009987', 'cellular process', 'IEA', 483, '96.8', '99.8'],
    ['GO:0034220', 'ion transmembrane transport', 'IEA', 483, '96.8', '99.8'],
    ['GO:0044765', 'single-organism transport', 'IEA', 483, '96.8', '99.8'],
    ['GO:0044763',
     'single-organism cellular process',
     'IEA',
     483,
   

In [169]:
pickle.dump(gos, open("737_eggnog_go.pkl", "wb"))

In [12]:
gos = pickle.load(open("737_eggnog_go.pkl", "rb"))

In [34]:
ggos = {k: v['go_terms'].get('Cellular Component', None) for k, v in gos.items()}
    

In [35]:
ggos_mf = {k: v['go_terms'].get('Molecular Function', None) for k, v in gos.items()}
    

In [36]:
ggos_bp = {k: v['go_terms'].get('Biological Process', None) for k, v in gos.items()}
    

In [45]:
ggos = {k:v for k,v in ggos.items() if v}
ggos_bp = {k:v for k,v in ggos_bp.items() if v}
ggos_mf = {k:v for k,v in ggos_mf.items() if v}

In [46]:
len(ggos), len(ggos_bp), len(ggos_mf)

(306, 543, 528)

In [47]:
goids={k:list(map(ig(0), v)) for k, v in ggos.items()}
goids_bp={k:list(map(ig(0), v)) for k, v in ggos_bp.items()}
goids_mf={k:list(map(ig(0), v)) for k, v in ggos_mf.items()}

In [41]:
sum(["GO:0016020" in v for k, v in goids.items()])

112

http://nbviewer.jupyter.org/urls/dessimozlab.github.io/go-handbook/GO%20Tutorial%20in%20Python%20-%20Solutions.ipynb

http://gohandbook.org/doku.php

In [42]:
list(map(lambda x: go[x], go['GO:0016020'].get_all_parents()))

[GOTerm('GO:0005575'):
   id:GO:0005575
   parents: 0 items
   is_obsolete:False
   depth:0
   name:cellular_component
   children: 22 items
   alt_ids: 1 items
     GO:0008372
   level:0
   namespace:cellular_component
   _parents: 0 items]

In [49]:
go_assoc = {k:list(map(ig(0),v)) for k,v in ggos.items()}
go_assoc_bp = {k:list(map(ig(0),v)) for k,v in ggos_bp.items()}
go_assoc_mf = {k:list(map(ig(0),v)) for k,v in ggos_mf.items()}

In [50]:
with open("737_go_cc_new.txt", "w") as fo:
    for k, v in go_assoc.items():
        fo.write("%s\t%s\n" %(k, "; ".join(v)))
        
with open("737_go_bp_new.txt", "w") as fo:
    for k, v in go_assoc_bp.items():
        fo.write("%s\t%s\n" %(k, "; ".join(v)))
        
with open("737_go_mf_new.txt", "w") as fo:
    for k, v in go_assoc_mf.items():
        fo.write("%s\t%s\n" %(k, "; ".join(v)))

## Enrichment & stuff ##

In [193]:
g = goatools.GOEnrichmentStudy(['new030_prec'],go_assoc,go, propagate_counts=False)

     1 out of      1 population items found in association


In [78]:
gm = pd.read_table("gene_association.ecocyc", skiprows=24, header=None)